In [ ]:
import math
import pandas as pd 
import numpy as np
import imblearn
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report, make_scorer, plot_roc_curve, recall_score, accuracy_score, precision_score
from sklearn.metrics import matthews_corrcoef 
from collections import Counter
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif

In [ ]:
!pip install bootstrapped

  Created wheel for bootstrapped: filename=bootstrapped-0.0.2-py2.py3-none-any.whl size=13954 sha256=4af20db3e35c43af73ddc48b3c83de2e1454dd07577cc83fad1e85c23db50ba2
  Stored in directory: /root/.cache/pip/wheels/15/55/6a/9a722f067ac4c3dfab359ed2ec7906b9cc6649156d9886bd59
Successfully built bootstrapped


In [ ]:
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving df.xlsx to df.xlsx


In [ ]:
df = pd.read_excel('df.xlsx')

In [ ]:
X=df.loc[:,['_1' in i for i in df.columns]]

In [ ]:
X = X.drop(["visit:data_visita_1", 'data_prelievo_1','ult_tsa:U_TSA_data_1',    #datetime variables
            'ana:istruzione_1',
            'ult_tsa:placca_dx_recod_1', 'ult_tsa:placca_sx_recod_1', 'ult_tsa:placca_1','ult_tsa:IMT_CC_max_sx_1', 'ult_tsa:IMT_CC_max_dx_1', 'ult_tsa:IMT_CC_medio_round_sx_1', 'ult_tsa:IMT_CC_medio_round_dx_1', 'ult_tsa:IMT_CC_medio_round_mean_1',  # variables regarding the presence of IMT and plaques
       'TG_cut_off_1', 'HDL_cut_off_1' , 'glucosio_cut_off_1' , 'pressione_cut_off_1', 'vita_guidelines_cut_off_1'  , 'vita_non_guidelines_cut_off_1' , 'sum_determinanti_guidelines_1',  'sum_determinanti_non_guidelines_1',  'mancanti_guidelines_1', 'mancanti_non_guidelines_1',   #other useless variables   
       'ana_pat:evento_1' ], axis=1)   #useless because at the time of the first visit no one has suffered from a cardiovascular event

In [ ]:
y=df['ult_tsa:placca_tot']
# summarize class distribution
counter = Counter(y)
print(counter)

Counter({0: 567, 1: 380})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state=42, stratify=y)


num_pipeline = Pipeline([
        ('scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('ohe', OneHotEncoder(handle_unknown = 'ignore')),
    ])
    
num_attribs = list( X_train.select_dtypes(include=['int64', 'float64']).columns)
cat_attribs = list( X_train.select_dtypes(include='object').columns)

preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
selector = SelectKBest(score_func=mutual_info_classif, k='all')
selector_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('selector', selector)]) 

In [ ]:
mcc_results=[]

DECISION TREE CLASSIFIER

In [ ]:
metrics_df_tree = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Recall", "Precision", 'MCC'])

In [ ]:
tree_param = {'model__max_leaf_nodes': list(range(2, 50)), 'model__min_samples_split': [2, 3, 4]}

steps = [('preprocessor', preprocessor), ('model', DecisionTreeClassifier(random_state=42))]
tree_pipe = Pipeline(steps=steps)



scorer = make_scorer(matthews_corrcoef)

In [ ]:

steps=[10, 25, 50, preprocessor.fit_transform(X_train).shape[1]] #None to have all the features selected 
n_ext=10
n_cv=5
ranking_tree = np.empty((n_ext * n_cv, preprocessor.fit_transform(X_train).shape[1]), dtype=int)

for n in range(n_ext):
    
    skf = StratifiedKFold(n_cv, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train_int, X_test_int = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_int, y_test_int = y_train.iloc[train_index], y_train.iloc[test_index]
        
        tuncv = StratifiedShuffleSplit(
                n_splits=n_cv, test_size=0.5, random_state=i
            )
        tree_grid = GridSearchCV(estimator = tree_pipe, param_grid = tree_param, scoring=scorer, cv=tuncv ,n_jobs=-1, verbose=False)

    
    
        tree_grid.fit(X_train_int ,y_train_int) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = tree_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train_int, y_train_int) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.scores_[selector.get_support()]
        #feature=list(zip(range(scores.shape[0]), scores))
        #feature.sort(key=lambda x:x[1])
        ranking_tmp = np.argsort(scores)[::-1]
        ranking_tree[(n * n_cv) + i] = ranking_tmp

    
        # rescaling
       
     
        X_train_int = preprocessor.fit_transform(X_train_int)
        X_test_int = preprocessor.transform(X_test_int)
        

        #for step in steps:
          #selected_features=[tupla[0] for tupla in feature[:step]]
          #X_train_fs, X_test_fs = X_train_int[:, selected_features], X_test_int[:, selected_features]
        for j, s in enumerate(steps):
          v = ranking_tree[(n * n_cv) + i][:s]
          X_train_fs, X_test_fs = X_train_int[:, v], X_test_int[:, v]
          best_model.fit(X_train_fs, y_train_int)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_tree  = metrics_df_tree.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": s,
                        "Recall": recall_score(y_test_int, y_pred),
                        "Precision": precision_score(y_test_int,y_pred) ,
                      
                          'MCC': matthews_corrcoef(y_test_int, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
metrics_df_tree.head()

,Iteration,Fold,Features,Recall,Precision,MCC
0,0.0,0.0,10.0,0.631579,0.62069,0.371721
1,0.0,0.0,25.0,0.631579,0.62069,0.371721
2,0.0,0.0,50.0,0.631579,0.62069,0.371721
3,0.0,0.0,NaN,0.631579,0.62069,0.371721
4,0.0,1.0,10.0,0.596491,0.68000,0.418987


In [ ]:
metrics_df_tree['Features'] = metrics_df_tree['Features'].fillna("all")

In [ ]:
metrics_df_tree.to_csv('metrics_df_tree.csv', encoding = 'utf-8-sig') 

In [ ]:
avg_df_tree=metrics_df_tree.groupby(['Features']).mean()

In [ ]:
avg_df_tree

,Unnamed: 0,Iteration,Fold,Recall,Precision,MCC
Features,,,,,,
10.0,98.0,4.5,2.0,0.537895,0.604456,0.318486
25.0,99.0,4.5,2.0,0.542456,0.600840,0.317098
50.0,100.0,4.5,2.0,0.541053,0.600045,0.314895
all,101.0,4.5,2.0,0.541754,0.599016,0.314494


In [ ]:
avg_df_tree.to_csv('avg_df_tree.csv', encoding = 'utf-8-sig') 

In [ ]:
avg_df_tree = pd.read_csv('avg_df_tree.csv')
avg_df_tree

,Features,Iteration,Fold,Recall,Precision,MCC
0,10.0,4.5,2.0,0.537895,0.604456,0.318486
1,25.0,4.5,2.0,0.542456,0.600840,0.317098
2,50.0,4.5,2.0,0.541053,0.600045,0.314895
3,all,4.5,2.0,0.541754,0.599016,0.314494


In [ ]:
#append confidence intervals

cols=['Recall' ,'Precision', 'MCC']

for col in cols:
  avg_df_tree[col]=metrics_df_tree.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_tree['Lower Bound '+ col]=metrics_df_tree.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_tree['Upper Bound '+ col]=metrics_df_tree.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)

In [ ]:
avg_df_tree.to_csv('CI_df_tree.csv', encoding = 'utf-8-sig') 

In [ ]:
CI_df_tree = pd.read_csv('CI_df_tree.csv')
CI_df_tree

,Features,Unnamed: 0,Iteration,Fold,Recall,Precision,MCC,Lower Bound Recall,Upper Bound Recall,Lower Bound Precision,Upper Bound Precision,Lower Bound MCC,Upper Bound MCC
0,10.0,98.0,4.5,2.0,0.537895,0.604456,0.318486,0.507719,0.572982,0.581373,0.636011,0.297035,0.341559
1,25.0,99.0,4.5,2.0,0.542456,0.600840,0.317098,0.514035,0.575789,0.577565,0.632175,0.293949,0.341190
2,50.0,100.0,4.5,2.0,0.541053,0.600045,0.314895,0.512281,0.573684,0.575872,0.632156,0.291040,0.339356
3,all,101.0,4.5,2.0,0.541754,0.599016,0.314494,0.512982,0.574386,0.575542,0.630481,0.290351,0.339333


In [ ]:
mcc_results.append (('Decision Tree ',
                 CI_df_tree[  (CI_df_tree.MCC == CI_df_tree.MCC.values.max())  ] ['MCC'].iloc[0],
                 CI_df_tree[  (CI_df_tree.MCC == CI_df_tree.MCC.values.max())  ]  ['Features'].iloc[0] ))

LOGISTIC REGRESSION


In [ ]:
metrics_df_log = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Recall", "Precision", 'MCC'])

In [ ]:
log_param = [
    {
    'model__penalty': ['l2', None],
    'model__C': np.logspace(-4, 4, 10),
    'model__solver': ['lbfgs'],
    'model__class_weight': [None, 'balanced']}
]


steps = [('preprocessor', preprocessor), ('model', LogisticRegression(random_state=42, max_iter=1000))]
log_pipe = Pipeline(steps=steps)

scorer = make_scorer(matthews_corrcoef)

In [ ]:
steps=[10, 25, 50, preprocessor.fit_transform(X_train).shape[1] ] 
n_ext=10
n_cv=5
ranking_log = np.empty((n_ext * n_cv, preprocessor.fit_transform(X_train).shape[1]), dtype=int)

for n in range(n_ext):
    
    skf = StratifiedKFold(n_cv, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train_int, X_test_int = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_int, y_test_int = y_train.iloc[train_index], y_train.iloc[test_index]
        
        tuncv = StratifiedShuffleSplit(
                n_splits=n_cv, test_size=0.5, random_state=i
            )
        log_grid = GridSearchCV(estimator = log_pipe, param_grid = log_param, scoring=scorer, cv=tuncv ,n_jobs=-1, verbose=False)

    
    
        log_grid.fit(X_train_int ,y_train_int) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = log_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train_int, y_train_int) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.scores_[selector.get_support()]
        #feature=list(zip(range(scores.shape[0]), scores))
        #feature.sort(key=lambda x:x[1])
        ranking_tmp = np.argsort(scores)[::-1]
        ranking_log[(n * n_cv) + i] = ranking_tmp

    
        # rescaling
       
     
        X_train_int = preprocessor.fit_transform(X_train_int)
        X_test_int = preprocessor.transform(X_test_int)
        

        #for step in steps:
          #selected_features=[tupla[0] for tupla in feature[:step]]
          #X_train_fs, X_test_fs = X_train_int[:, selected_features], X_test_int[:, selected_features]
        for j, s in enumerate(steps):
          v = ranking_log[(n * n_cv) + i][:s]
          X_train_fs, X_test_fs = X_train_int[:, v], X_test_int[:, v]
          best_model.fit(X_train_fs, y_train_int)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_log  = metrics_df_log.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": s,
                        "Recall": recall_score(y_test_int, y_pred),
                        "Precision": precision_score(y_test_int,y_pred) ,
                      
                          'MCC': matthews_corrcoef(y_test_int, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 266, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)


In [ ]:
metrics_df_log.head()

,Iteration,Fold,Features,Recall,Precision,MCC
0,0.0,0.0,10.0,0.649123,0.616667,0.375638
1,0.0,0.0,25.0,0.614035,0.593220,0.329916
2,0.0,0.0,50.0,0.578947,0.600000,0.322125
3,0.0,0.0,67.0,0.596491,0.576271,0.300764
4,0.0,1.0,10.0,0.403509,0.676471,0.314847


In [ ]:
metrics_df_log.to_csv('metrics_df_log.csv', encoding = 'utf-8-sig') 

In [ ]:
metrics_df_log=pd.read_csv('metrics_df_log.csv')

In [ ]:
avg_df_log=metrics_df_log.groupby(['Features']).mean()

In [ ]:
avg_df_log.to_csv('avg_df_log.csv', encoding = 'utf-8-sig') 

In [ ]:
avg_df_log = pd.read_csv('avg_df_log.csv')

In [ ]:
avg_df_log

,Iteration,Fold,Recall,Precision,MCC
Features,,,,,
10.0,4.5,2.0,0.568070,0.582732,0.290096
25.0,4.5,2.0,0.571930,0.571216,0.280747
50.0,4.5,2.0,0.574737,0.576907,0.288158
67.0,4.5,2.0,0.578947,0.575601,0.288023


In [ ]:
#append confidence intervals
cols=['Recall' ,'Precision', 'MCC']

for col in cols:
  avg_df_log[col]=metrics_df_log.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_log['Lower Bound '+ col]=metrics_df_log.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_log['Upper Bound '+ col]=metrics_df_log.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)



In [ ]:
avg_df_log.to_csv('CI_df_log.csv', encoding = 'utf-8-sig')

In [ ]:
CI_df_log = pd.read_csv('CI_df_log.csv')
CI_df_log

,Features,Iteration,Fold,Recall,Precision,MCC,Lower Bound Recall,Upper Bound Recall,Lower Bound Precision,Upper Bound Precision,Lower Bound MCC,Upper Bound MCC
0,10.0,4.5,2.0,0.568070,0.582732,0.290096,0.535088,0.603509,0.565351,0.599195,0.269789,0.310596
1,25.0,4.5,2.0,0.571930,0.571216,0.280747,0.539649,0.605263,0.556214,0.585607,0.258358,0.302657
2,50.0,4.5,2.0,0.574737,0.576907,0.288158,0.545614,0.605263,0.560984,0.592228,0.266219,0.309070
3,67.0,4.5,2.0,0.578947,0.575601,0.288023,0.550877,0.607368,0.559341,0.590883,0.268053,0.307954


In [ ]:
mcc_results.append (('Logistic Regression ',
                 CI_df_log[  (CI_df_log.MCC == CI_df_log.MCC.values.max())  ] ['MCC'].iloc[0],
                 CI_df_log[  (CI_df_log.MCC == CI_df_log.MCC.values.max())  ]  ['Features'].iloc[0] ))

KNN

In [ ]:
metrics_df_knn = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Recall", "Precision", 'MCC'])

In [ ]:
knn_param= {'model__n_neighbors': np.arange(1, 25)}

steps = [('preprocessor', preprocessor), ('model', KNeighborsClassifier())]
knn_pipe = Pipeline(steps=steps)

scorer = make_scorer(matthews_corrcoef)

In [ ]:
steps=[10, 25, 50, preprocessor.fit_transform(X_train).shape[1] ] 
n_ext=10
n_cv=5
ranking_knn = np.empty((n_ext * n_cv, preprocessor.fit_transform(X_train).shape[1]), dtype=int)

for n in range(n_ext):
    
    skf = StratifiedKFold(n_cv, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train_int, X_test_int = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_int, y_test_int = y_train.iloc[train_index], y_train.iloc[test_index]
        
        tuncv = StratifiedShuffleSplit(
                n_splits=n_cv, test_size=0.5, random_state=i
            )
        knn_grid = GridSearchCV(estimator = knn_pipe, param_grid = knn_param, scoring=scorer, cv=tuncv ,n_jobs=-1, verbose=False)

    
    
        knn_grid.fit(X_train_int ,y_train_int) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = knn_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train_int, y_train_int) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.scores_[selector.get_support()]
        #feature=list(zip(range(scores.shape[0]), scores))
        #feature.sort(key=lambda x:x[1])
        ranking_tmp = np.argsort(scores)[::-1]
        ranking_knn[(n * n_cv) + i] = ranking_tmp

    
        # rescaling
       
     
        X_train_int = preprocessor.fit_transform(X_train_int)
        X_test_int = preprocessor.transform(X_test_int)
        

        #for step in steps:
          #selected_features=[tupla[0] for tupla in feature[:step]]
          #X_train_fs, X_test_fs = X_train_int[:, selected_features], X_test_int[:, selected_features]
        for j, s in enumerate(steps):
          v = ranking_knn[(n * n_cv) + i][:s]
          X_train_fs, X_test_fs = X_train_int[:, v], X_test_int[:, v]
          best_model.fit(X_train_fs, y_train_int)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_knn  = metrics_df_knn.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": s,
                        "Recall": recall_score(y_test_int, y_pred),
                        "Precision": precision_score(y_test_int,y_pred) ,
                      
                          'MCC': matthews_corrcoef(y_test_int, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )

In [ ]:
metrics_df_knn.to_csv('metrics_df_knn.csv', encoding = 'utf-8-sig') 

In [ ]:
metrics_df_knn=pd.read_csv('metrics_df_knn.csv')

In [ ]:
avg_df_knn=metrics_df_knn.groupby(['Features']).mean()

In [ ]:
avg_df_knn

,Unnamed: 0,Iteration,Fold,Recall,Precision,MCC
Features,,,,,,
10.0,98.0,4.5,2.0,0.454386,0.581748,0.246340
25.0,99.0,4.5,2.0,0.440351,0.575396,0.234117
50.0,100.0,4.5,2.0,0.402807,0.594390,0.238144
67.0,101.0,4.5,2.0,0.401404,0.582244,0.225311


In [ ]:
avg_df_knn.to_csv('avg_df_knn.csv', encoding = 'utf-8-sig')

In [ ]:
#append confidence intervals
cols=['Recall' ,'Precision', 'MCC']

for col in cols:
  avg_df_knn[col]=metrics_df_knn.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_knn['Lower Bound '+ col]=metrics_df_knn.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_knn['Upper Bound '+ col]=metrics_df_knn.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)

In [ ]:
avg_df_knn.to_csv('CI_df_knn.csv', encoding = 'utf-8-sig') 

In [ ]:
CI_df_knn = pd.read_csv('CI_df_knn.csv')
CI_df_knn

,Features,Unnamed: 0,Iteration,Fold,Recall,Precision,MCC,Lower Bound Recall,Upper Bound Recall,Lower Bound Precision,Upper Bound Precision,Lower Bound MCC,Upper Bound MCC
0,10.0,98.0,4.5,2.0,0.454386,0.581748,0.246340,0.432982,0.476500,0.563768,0.599851,0.222152,0.270300
1,25.0,99.0,4.5,2.0,0.440351,0.575396,0.234117,0.417895,0.462807,0.556111,0.593834,0.209563,0.257734
2,50.0,100.0,4.5,2.0,0.402807,0.594390,0.238144,0.383860,0.422456,0.573152,0.614885,0.213666,0.261961
3,67.0,101.0,4.5,2.0,0.401404,0.582244,0.225311,0.380702,0.422456,0.567248,0.596588,0.208408,0.241905


In [ ]:
mcc_results.append (('KNN',
                 CI_df_knn[  (CI_df_knn.MCC == CI_df_knn.MCC.values.max())  ] ['MCC'].iloc[0],
                 CI_df_knn[  (CI_df_knn.MCC == CI_df_knn.MCC.values.max())  ]  ['Features'].iloc[0] ))

RANDOM FOREST 


In [ ]:
metrics_df_rf = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Recall", "Precision", 'MCC'])


In [ ]:
rf_param =  {
    
    'model__max_depth': [1,5,10, 15],
    'model__max_features': ['auto', 'log2' ,2],
    'model__n_estimators': [100, 200, 500]
}

steps = [('preprocessor', preprocessor), ('model', RandomForestClassifier())]
rf_pipe = Pipeline(steps=steps)

scorer = make_scorer(matthews_corrcoef)

In [ ]:
steps=[10, 25, 50, preprocessor.fit_transform(X_train).shape[1] ] 
n_ext=10
n_cv=5
ranking_rf = np.empty((n_ext * n_cv, preprocessor.fit_transform(X_train).shape[1]), dtype=int)

for n in range(n_ext):
    
    skf = StratifiedKFold(n_cv, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train_int, X_test_int = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_int, y_test_int = y_train.iloc[train_index], y_train.iloc[test_index]
        
        tuncv = StratifiedShuffleSplit(
                n_splits=n_cv, test_size=0.5, random_state=i
            )
        rf_grid = GridSearchCV(estimator = rf_pipe, param_grid = rf_param, scoring=scorer, cv=tuncv ,n_jobs=-1, verbose=False)

    
    
        rf_grid.fit(X_train_int ,y_train_int) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = rf_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train_int, y_train_int) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.scores_[selector.get_support()]
        #feature=list(zip(range(scores.shape[0]), scores))
        #feature.sort(key=lambda x:x[1])
        ranking_tmp = np.argsort(scores)[::-1]
        ranking_rf[(n * n_cv) + i] = ranking_tmp

    
        # rescaling
       
     
        X_train_int = preprocessor.fit_transform(X_train_int)
        X_test_int = preprocessor.transform(X_test_int)
        

        #for step in steps:
          #selected_features=[tupla[0] for tupla in feature[:step]]
          #X_train_fs, X_test_fs = X_train_int[:, selected_features], X_test_int[:, selected_features]
        for j, s in enumerate(steps):
          v = ranking_knn[(n * n_cv) + i][:s]
          X_train_fs, X_test_fs = X_train_int[:, v], X_test_int[:, v]
          best_model.fit(X_train_fs, y_train_int)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_rf  = metrics_df_rf.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": s,
                        "Recall": recall_score(y_test_int, y_pred),
                        "Precision": precision_score(y_test_int,y_pred) ,
                      
                          'MCC': matthews_corrcoef(y_test_int, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )

KeyboardInterrupt: ignored

In [ ]:
len(metrics_df_rf)


148

In [ ]:
metrics_df_rf['Features'] = metrics_df_rf['Features'].fillna("all")

In [ ]:
metrics_df_rf.to_csv('metrics_df_rf.csv', encoding = 'utf-8-sig') 

In [ ]:
metrics_df_rf=pd.read_csv('metrics_df_rf.csv')

In [ ]:
avg_df_rf=metrics_df_rf.groupby(['Features']).mean()

In [ ]:
avg_df_rf.to_csv('avg_df_rf.csv', encoding = 'utf-8-sig') 

In [ ]:
avg_df_rf

,Iteration,Fold,Recall,Precision,MCC
Features,,,,,
10.0,3.216216,1.918919,0.502134,0.621884,0.310525
25.0,3.216216,1.918919,0.500711,0.646618,0.332822
50.0,3.216216,1.918919,0.487909,0.659329,0.340846
67.0,3.216216,1.918919,0.481271,0.666049,0.343064


In [ ]:
#append confidence intervals
cols=['Recall' ,'Precision', 'MCC']

for col in cols:
  avg_df_rf[col]=metrics_df_rf.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_rf['Lower Bound '+ col]=metrics_df_rf.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_rf['Upper Bound '+ col]=metrics_df_rf.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)

In [ ]:
avg_df_rf.to_csv('CI_df_rf.csv', encoding = 'utf-8-sig') 

In [ ]:
CI_df_rf = pd.read_csv('CI_df_rf.csv')
CI_df_rf

,Features,Iteration,Fold,Recall,Precision,MCC,Lower Bound Recall,Upper Bound Recall,Lower Bound Precision,Upper Bound Precision,Lower Bound MCC,Upper Bound MCC
0,10.0,3.216216,1.918919,0.502134,0.621884,0.310525,0.477952,0.526790,0.600790,0.640898,0.283657,0.336176
1,25.0,3.216216,1.918919,0.500711,0.646618,0.332822,0.478900,0.522997,0.624652,0.668461,0.307025,0.358565
2,50.0,3.216216,1.918919,0.487909,0.659329,0.340846,0.464675,0.510669,0.636963,0.680740,0.314149,0.367175
3,67.0,3.216216,1.918919,0.481271,0.666049,0.343064,0.453770,0.508298,0.641180,0.690823,0.312675,0.373849


In [ ]:
mcc_results.append (('Random Forest',
                 CI_df_rf[  (CI_df_rf.MCC == CI_df_rf.MCC.values.max())  ] ['MCC'].iloc[0],
                 CI_df_rf[  (CI_df_rf.MCC == CI_df_rf.MCC.values.max())  ]  ['Features'].iloc[0] ))

CONCLUSIONS

In [ ]:
mcc_results 

[('Logistic Regression ', 0.2900961417417739, 10.0),
 ('KNN', 0.2463404281570071, 10.0),
 ('Random Forest', 0.3430637966324857, 67.0),
 ('Decision Tree ', 0.318485976770124, '10.0')]

In [ ]:
#final model
from utilities import *
print(final_model(mcc_results))

{'model': 'Random Forest', 'metric_value': 0.3430637966324857, 'features': 67.0}


VALIDATION EXPERIMENT


In [ ]:
rf_param =  {
    
    'model__max_depth': [1,5,10, 15],
    'model__max_features': ['auto', 'log2' ,2],
    'model__n_estimators': [100, 200, 500]
}

steps = [('preprocessor', preprocessor), ('model', RandomForestClassifier())]
rf_pipe = Pipeline(steps=steps)

scorer = make_scorer(matthews_corrcoef)
cv = StratifiedShuffleSplit(
                n_splits=n_cv, test_size=0.5, random_state=123
            )
rf_grid = GridSearchCV(estimator = rf_pipe, param_grid = rf_param, cv=cv,  scoring=make_scorer(matthews_corrcoef) ,n_jobs=-1, verbose=False)

In [ ]:
rf_grid.fit(X_train, y_train) #GridSearchCV over training  fold to find the optimal parameters on the entire train and test set  

y_pred=rf_grid.predict(X_test)

print('mcc%.3f' %matthews_corrcoef(y_test, y_pred))

mcc0.285


Compare this feature selection with the one operated by the authors of the study:

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving df_newfilter.xlsx to df_newfilter.xlsx


In [ ]:
df_newfilter= pd.read_excel('df_newfilter.xlsx')

In [ ]:
X = df_newfilter.drop(['ult_tsa:placca_tot','ult_tsa:placca_dx_recod_1', 'ult_tsa:placca_sx_recod_1', 'ult_tsa:placca_1','ult_tsa:IMT_CC_max_sx_1', 'ult_tsa:IMT_CC_max_dx_1', 'ult_tsa:IMT_CC_medio_round_sx_1', 'ult_tsa:IMT_CC_medio_round_dx_1', 'ult_tsa:IMT_CC_medio_round_mean_1'], axis=1)

In [ ]:
y=df_newfilter['ult_tsa:placca_tot']
# summarize class distribution
counter = Counter(y)
print(counter)

Counter({0: 557, 1: 374})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=42)

num_pipeline = Pipeline([
        ('scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('ohe', OneHotEncoder(handle_unknown = 'ignore')),
    ])
num_attribs = list( X_train.select_dtypes(include=['int64', 'float64']).columns)
cat_attribs = list( X_train.select_dtypes(include='object').columns)

preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
rf_param =  {
    
    'model__max_depth': [1,5,10, 15],
    'model__max_features': ['auto', 'log2' ,2],
    'model__n_estimators': [100, 200, 500]
}

steps = [('preprocessor', preprocessor), ('model', RandomForestClassifier())]
rf_pipe = Pipeline(steps=steps)

scorer = make_scorer(matthews_corrcoef)
cv = StratifiedShuffleSplit(
                n_splits=n_cv, test_size=0.5, random_state=123
            )
rf_grid = GridSearchCV(estimator = rf_pipe, param_grid = rf_param, cv=cv,  scoring=make_scorer(matthews_corrcoef) ,n_jobs=-1, verbose=False)

In [ ]:
rf_grid.fit(X_train, y_train) #GridSearchCV over training  fold to find the optimal parameters on the entire train and test set  

y_pred=rf_grid.predict(X_test)

print('mcc according to this new filtering :%3f' %matthews_corrcoef(y_test, y_pred))

mcc according to this new filtering :0.270641
